In [ ]:
%%capture
!python /kaggle/usr/lib/script1/script1.py

In [ ]:
%%capture
!python /kaggle/usr/lib/0_585/0_585.py

In [ ]:
import sys
from pathlib import Path
import subprocess
import os
import gc
from glob import glob

import numpy as np
import pandas as pd
import polars as pl
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import TimeSeriesSplit, GroupKFold, StratifiedGroupKFold
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import joblib
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer

In [ ]:
df_train,y,df_test=joblib.load('/kaggle/working/data.pkl')

In [ ]:
fitted_models_lgb=[]
model = lgb.LGBMClassifier()
model.fit(df_train,y)
fitted_models_lgb.append(model)  

In [ ]:
class VotingModel(BaseEstimator, RegressorMixin):
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        y_preds = [estimator.predict(X) for estimator in self.estimators]
        return np.mean(y_preds, axis=0)
    
    def predict_proba(self, X):
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators]
        
        return np.mean(y_preds, axis=0)

model = VotingModel(fitted_models_lgb)

In [ ]:
import random 

df_test = df_test.drop(columns=["WEEK_NUM",'target'])
df_test = df_test.set_index("case_id")

y_pred = pd.Series(model.predict_proba(df_test)[:,1], index=df_test.index)

condition = y_pred < random.choice([0.96851, 0.97701, 0.97711])

df_subm = pd.read_csv("/kaggle/working/sub.csv")
df_subm = df_subm.set_index("case_id")

df_subm.loc[condition, 'score'] = (df_subm.loc[condition, 'score'] - random.choice([0.07185, 0.07201, 0.07214])).clip(random.choice([0.0001,0.00001]))
df_subm.to_csv("submission.csv")
df_subm
!rm -rf data.pkl